In [148]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import json

In [149]:
#pip install python-dotenv

In [151]:
PURPLEAIR_GROUP_ID = os.getenv('PURPLEAIR_GROUP_ID')
PURPLEAIR_READ_KEY = os.getenv('PURPLEAIR_READ_KEY')
PURPLEAIR_WRITE_KEY = os.getenv('PURPLEAIR_WRITE_KEY')

In [152]:
def consulta_sensores():    
    api_url = f'https://api.purpleair.com/v1/groups/{PURPLEAIR_GROUP_ID}'
    headers = {'X-API-Key': PURPLEAIR_READ_KEY}
    response = requests.get(api_url, headers=headers)
    members = response.json()['members']
    sensor_index = [ sub['sensor_index'] for sub in members ]
    member_id = [ sub['id'] for sub in members ]
    return sensor_index, member_id

In [153]:
def adicionar_sensor(sensor_index):    
    api_url = f'https://api.purpleair.com/v1/groups/{PURPLEAIR_GROUP_ID}/members'
    headers = {'Content-Type': 'application/json','X-API-Key': PURPLEAIR_WRITE_KEY}
    data = {'sensor_index': sensor_index}
    response = requests.post(api_url, data=json.dumps(data), headers=headers)
    return response.status_code

In [154]:
def eliminar_sensor(member_id):    
    api_url = f'https://api.purpleair.com/v1/groups/{PURPLEAIR_GROUP_ID}/members/{member_id}'
    headers = {'Content-Type': 'application/json','X-API-Key': PURPLEAIR_WRITE_KEY}
    response = requests.delete(api_url, headers=headers)
    return response.status_code

In [155]:
def actualizaciones(df_actualizar, df_actual):
    actualizar = pd.merge(df_actualizar,df_actual,how="outer",on="sensor_index",indicator=True)
    control = actualizar.query('_merge!="both"')
    adicionar = actualizar.query('_merge=="left_only"')['sensor_index']
    eliminar = actualizar.query('_merge=="right_only"')[['sensor_index','member_id']]
    return control, adicionar, eliminar

In [156]:
sensores_actualizar = pd.read_json('sensores.json')

sensor_index, member_id = consulta_sensores()
sensores_actuales = pd.DataFrame({'sensor_index': sensor_index, 'member_id': member_id})

control, adicionar, eliminar = actualizaciones(sensores_actualizar,sensores_actuales)

if control.size > 0:
    if adicionar.size > 0:
        for sensor in adicionar:
            print('### Adicionando:',sensor)
            adicionar_sensor(sensor)
    elif eliminar.size > 0:
        for index, row in eliminar.iterrows():
            print('### Eliminando member_id:',row['member_id'],'| sensor_index:',row['sensor_index'])
            eliminar_sensor(row['member_id'])
else:
    print('### No hay actualizaciones')

### No hay actualizaciones
